# RNN from scratch

In [18]:
import glob
import torch
from importlib import reload

%load_ext autoreload
%autoreload 2
from utils.files import *

%load_ext autoreload
%autoreload 2
from src.RNN import RNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data preparation

In [19]:
article_paths = glob.glob("data/articles/*.txt")

corpus = [
    read_file(article_path) for article_path in article_paths
]

# Wikipedia loves non-ascii charachters, the vocab will skyrocket if I didn't limit it to ascii
corpus = [
    ''.join(c.lower() for c in article if ord(c) < 128) for article in corpus
]
print(corpus[1])
print("number of articles:", len(corpus))

the american civil war (april 12, 1861  may 26, 1865; also known by other names) was a civil war in the united states between the union ("the north") and the confederacy ("the south"), which was formed in 1861 by states that had seceded from the union. the central conflict leading to war was a dispute over whether slavery should be permitted to expand into the western territories, leading to more slave states, or be prohibited from doing so, which many believed would place slavery on a course of ultimate extinction. decades of controversy over slavery came to a head when abraham lincoln, who opposed slavery's expansion, won the 1860 presidential election. seven southern slave states responded to lincoln's victory by seceding from the united states and forming the confederacy. the confederacy seized u.s. forts and other federal assets within their borders. the war began on april 12, 1861, when the confederacy bombarded fort sumter in south carolina. a wave of enthusiasm for war swept ov

In [20]:
full_text = ' '.join(corpus)
unique_characters = sorted(list(set(full_text)))

In [21]:
# Picking an eos token & making sure it's not there
eos = "<eos>"
if eos in unique_characters:
    print("not suitable")
else:
    print("suitable")
    unique_characters.append(eos)

suitable


In [22]:
input_size = len(unique_characters)
print(f"Vocab size: {input_size}")

Vocab size: 59


In [23]:
token_to_index = {ch: i for i, ch in enumerate(unique_characters)}
index_to_token = {i: ch for i, ch in enumerate(unique_characters)}

In [24]:
sequences = []
for article in corpus:
    # Convert article characters to token indices
    sequence = [token_to_index[token] for token in article]
    # Append the end-of-sequence token index
    sequence.append(token_to_index[eos])
    # Add the complete sequence to the list of sequences
    sequences.append(sequence)

print(sequences[0])

[31, 36, 37, 38, 31, 44, 39, 49, 50, 31, 44, 12, 0, 45, 36, 36, 39, 33, 39, 31, 42, 42, 55, 0, 50, 38, 35, 0, 39, 49, 42, 31, 43, 39, 33, 0, 35, 43, 39, 48, 31, 50, 35, 0, 45, 36, 0, 31, 36, 37, 38, 31, 44, 39, 49, 50, 31, 44, 12, 0, 39, 49, 0, 31, 0, 42, 31, 44, 34, 42, 45, 33, 41, 35, 34, 0, 33, 45, 51, 44, 50, 48, 55, 0, 42, 45, 33, 31, 50, 35, 34, 0, 31, 50, 0, 50, 38, 35, 0, 33, 48, 45, 49, 49, 48, 45, 31, 34, 49, 0, 45, 36, 0, 33, 35, 44, 50, 48, 31, 42, 0, 31, 49, 39, 31, 0, 31, 44, 34, 0, 49, 45, 51, 50, 38, 0, 31, 49, 39, 31, 14, 0, 39, 50, 0, 39, 49, 0, 32, 45, 48, 34, 35, 48, 35, 34, 0, 32, 55, 0, 46, 31, 41, 39, 49, 50, 31, 44, 0, 50, 45, 0, 50, 38, 35, 0, 35, 31, 49, 50, 0, 31, 44, 34, 0, 49, 45, 51, 50, 38, 12, 0, 39, 48, 31, 44, 0, 50, 45, 0, 50, 38, 35, 0, 53, 35, 49, 50, 12, 0, 50, 51, 48, 41, 43, 35, 44, 39, 49, 50, 31, 44, 0, 50, 45, 0, 50, 38, 35, 0, 44, 45, 48, 50, 38, 53, 35, 49, 50, 12, 0, 51, 56, 32, 35, 41, 39, 49, 50, 31, 44, 0, 50, 45, 0, 50, 38, 35, 0, 44, 4

In [25]:
input_size = len(unique_characters)
hidden_size = 30
output_size = len(unique_characters)

In [26]:
model = RNN(input_size=input_size, hidden_size=hidden_size, output_size=output_size,
            token_to_index=token_to_index, index_to_token=index_to_token,
            index_to_embedding=None, one_hot=True, eos=eos)

In [27]:
# shorties_indexes = torch.argsort(torch.tensor([len(sequence) for sequence in sequences]))
# shorties = []
# for index in shorties_indexes[:10]:
#     shorties.append(sequences[index])

In [28]:
# print(shorties[3])

In [29]:
text_example = "The U.S is the root of all evil in the".lower()
encoded = [token_to_index[token] for token in text_example]
output = model.generate(sequence=encoded, max_length=100, deterministic=True)
print(output)

Tokens Generated::   0%|          | 0/100 [00:00<?, ?it/s]

????????????????????????????????????????????????????????????????????????????????????????????????????


In [30]:
shorties = [
    sequence[:100] for sequence in sequences[:1]
]

In [31]:
checkpoints = model.train(sequences=shorties, epochs=2000,
                          checkpoints=[500, 1000, 2000])

Epochs Finished:   0%|          | 0/2000 [00:00<?, ?it/s]

Sequences Finished::   0%|          | 0/1 [00:00<?, ?it/s]

Sequences Finished::   0%|          | 0/1 [00:00<?, ?it/s]

Sequences Finished::   0%|          | 0/1 [00:00<?, ?it/s]

Sequences Finished::   0%|          | 0/1 [00:00<?, ?it/s]

...

Sequences Finished::   0%|          | 0/1 [00:00<?, ?it/s]

Sequences Finished::   0%|          | 0/1 [00:00<?, ?it/s]

Sequences Finished::   0%|          | 0/1 [00:00<?, ?it/s]

Sequences Finished::   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 2000 Loss: 0.49983553029596806
Model Saved @ Epoch 2000


In [32]:
print(
    ''.join([index_to_token[index] for index in shorties[0]])
)

afghanistan, officially the islamic emirate of afghanistan, is a landlocked country located at the c


In [33]:
for i in range(3):
    current_model = checkpoints[i]
    text_example = "afghanistan".lower()
    encoded = [token_to_index[token] for token in text_example]
    output = current_model.generate(sequence=encoded, max_length=100)
    print(f"Experiment with {i} iterations:\n{output}\n\n")

Tokens Generated::   0%|          | 0/100 [00:00<?, ?it/s]

Experiment with 0 iterations:
ee cadaeo t i, uloae   eh fsnemoh nt  h eha,nieet    l ctaaca gli  auni i mud  ahuca ud niitthucia o




Tokens Generated::   0%|          | 0/100 [00:00<?, ?it/s]

Experiment with 1 iterations:
tn, c utemirt, counisthmicaf  age the thanilanirfd,,lefiraniathanisten,cate ahe isiate e , offiuafun




Tokens Generated::   0%|          | 0/100 [00:00<?, ?it/s]

Experiment with 2 iterations:
, officiamic amirad lococate afgh rfghanistan the c landlffician, the cry aha at officiamic anisthe 




# Word tokens

In [34]:
import nltk
from nltk.tokenize import word_tokenize
import gensim.downloader as api


# Ensure you have the 'punkt' tokenizer models downloaded for nltk
# try:
#     word_tokenize("example text")
# except LookupError:
#     nltk.download('punkt')

# Load pre-trained Word2Vec model
try:
    wv_model = api.load('word2vec-google-news-300')
    print("Gensim Word2Vec model loaded successfully.")
except Exception as e:
    print(f"Error loading Gensim Word2Vec model: {e}")
    print("Please ensure you have an internet connection to download the model or have it cached.")
    wv_model = None


# Tokenize the full_text into words
words = word_tokenize(full_text)
words.append(eos)
unique_words = sorted(list(set(words)))

word_to_index = {word: i for i, word in enumerate(unique_words)}
index_to_word = {i: word for i, word in enumerate(unique_words)}

print(f"Number of unique words: {len(unique_words)}")


word_to_embedding = {}
if wv_model:
    for word in unique_words:
        if word in wv_model:
            word_to_embedding[word] = wv_model[word]
        else:
            # Handle out-of-vocabulary words, e.g., by assigning a zero vector or random vector
            # For simplicity, we'll skip them here or you can assign a default
            # word_to_embedding[word] = np.zeros(wv_model.vector_size) # Requires numpy import
            pass 
    if 'example' in word_to_embedding:
        print(f"Embedding for the word 'example': {word_to_embedding.get('example', 'Not in vocab')}")
    else:
        print(f"Embedding for the word 'example': Not in vocab or model")
    print(f"Embedding dimension: {wv_model.vector_size}")
else:
    print("Gensim Word2Vec model not loaded. Cannot generate embeddings.")

# Example: Get embedding for a specific word
# if wv_model and 'example' in word_to_embedding:
#     print(f"Embedding for 'example': {word_to_embedding['example'][:10]}") # Print first 10 dims


ImportError: cannot import name 'triu' from 'scipy.linalg' (c:\Users\kingk\PycharmProjects\RNN_From_Scratch\.venv\Lib\site-packages\scipy\linalg\__init__.py)